CARGAR LOS DATOS

In [2]:
import pandas as pd

orders  = pd.read_csv(r'data\orders.csv')
order_products_prior = pd.read_csv(r'data\order_products__prior.csv')
order_products_train = pd.read_csv(r'data\order_products__train.csv')
products = pd.read_csv(r'data\products.csv')

print(orders.columns)
print(order_products_prior.columns)
print(order_products_train.columns)
print(products.columns)

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')


HACER UN MERGE DE LOS CSV

In [3]:
order_products = pd.concat([order_products_prior, order_products_train])

order_products = order_products.merge(products, on="product_id", how="left")

order_products = order_products.merge(orders[['order_id', 'order_dow', 'order_hour_of_day']], on="order_id", how="left")

order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,order_dow,order_hour_of_day
0,2,33120,1,1,Organic Egg Whites,86,16,5,9
1,2,28985,2,1,Michigan Organic Kale,83,4,5,9
2,2,9327,3,0,Garlic Powder,104,13,5,9
3,2,45918,4,1,Coconut Butter,19,13,5,9
4,2,30035,5,0,Natural Sweetener,17,13,5,9


FILTRAR LOS PRODUCTOS MÁS POPULARES

In [4]:
# Contar cuántas veces aparece cada producto
product_counts = order_products['product_id'].value_counts()

# Filtrar productos que aparecen en al menos 500 compras (puedes ajustar este número)
popular_products = product_counts[product_counts > 500].index

# Filtrar el dataset para trabajar solo con los productos más comprados
order_products_filtered = order_products[order_products['product_id'].isin(popular_products)]

LIMITAR EL NÚMERO DE TRANSACCIONES

In [5]:
# Tomar una muestra de 500,000 órdenes para reducir memoria
sample_orders = order_products_filtered['order_id'].sample(n=100000, random_state=42)

# Filtrar las órdenes en la muestra
order_sample = order_products_filtered[order_products_filtered['order_id'].isin(sample_orders)]

CREAR LA MATRIZ DE TRANSACCIONES CON One-Hot Encoding

In [6]:
from scipy.sparse import coo_matrix

# Eliminar duplicados (producto por orden)
order_sample_filtered = order_sample.drop_duplicates(subset=['order_id', 'product_name'])

# Crear la matriz dispersa (sparse matrix)
product_to_id = {product: idx for idx, product in enumerate(order_sample_filtered['product_name'].unique())}
order_sample_filtered['product_id_num'] = order_sample_filtered['product_name'].map(product_to_id)

row = order_sample_filtered['order_id'].astype('category').cat.codes
col = order_sample_filtered['product_id_num']
data = [1] * len(order_sample_filtered)

# Crear la matriz dispersa
basket_sparse = coo_matrix((data, (row, col)), shape=(order_sample_filtered['order_id'].nunique(), len(product_to_id)))

APLICAR EL ALGORITMO APRIORI

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

basket_df = pd.DataFrame.sparse.from_spmatrix(basket_sparse, columns=product_to_id.keys())

# Convertir los valores de la matriz en binarios (1 si el producto fue comprado, 0 si no)
basket_df = basket_df.applymap(lambda x: 1 if x > 0 else 0)

# Aplicar Apriori para obtener los conjuntos frecuentes
frequent_itemsets = apriori(basket_df, min_support=0.01, use_colnames=True)

# Generar reglas de asociación
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

C:\Users\joant\AppData\Local\Temp\ipykernel_4572\691805405.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_df = basket_df.applymap(lambda x: 1 if x > 0 else 0)
C:\Users\joant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


FILTRAR REGLAS POR LAS MAS CONFIABLES

In [12]:
confidence = 0.25

rules_sorted = rules.sort_values(by=['support', 'confidence', 'lift'], ascending=[False, False, False])

rules_filtered = rules_sorted[rules_sorted['confidence'] > confidence]


rules_filtered.to_csv(f'filtered_rules_{confidence}.csv',index=False)
rules_sorted.to_csv('sorted_rules.csv',index=False)